In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
covid = pd.read_csv("C:/Users/kosti/OneDrive/Desktop/School/Econ Classes/EC 499/Data/Strigency/Strigency.csv")
gdp = pd.read_csv("C:/Users/kosti/OneDrive/Desktop/School/Econ Classes/EC 499/Data/GDP/AllStateGDP.csv")
energy = pd.read_excel("C:/Users/kosti/OneDrive/Desktop/UMICH/SI/SI 671/Project/HS861M 2010-.xlsx")


In [3]:
# date is in format YYYY-MM-DD

gdp['Date'] = pd.to_datetime(gdp['DATE'])

In [4]:
# Removing rows where 'Year' or 'Month' columns contain non-numeric data
data_monthly_states_cleaned = energy[pd.to_numeric(energy['Year'], errors='coerce').notnull()]
data_monthly_states_cleaned = data_monthly_states_cleaned[pd.to_numeric(data_monthly_states_cleaned['Month'], errors='coerce').notnull()]

# Convert 'Year' and 'Month' columns to integer to prepare for combining them into a date
data_monthly_states_cleaned['Year'] = data_monthly_states_cleaned['Year'].astype(int)
data_monthly_states_cleaned['Month'] = data_monthly_states_cleaned['Month'].astype(int)

# Combining 'Year' and 'Month' into a single 'Date' column
data_monthly_states_cleaned['Date'] = pd.to_datetime(data_monthly_states_cleaned[['Year', 'Month']].assign(DAY=1))

energy = data_monthly_states_cleaned


In [5]:
features = ['State', 'Date', 'Residential Sales', 'Commercial Sales', 'Industrial Sales', 'Transportation Sales']
energy = energy[features]

covid['Date'] = pd.to_datetime(covid[['Year', 'Month']].assign(DAY=1))

# take last two letters of region code to get state
covid['State'] = covid['RegionCode'].str[-2:]

#fill in missing values with 0
covid = covid.fillna(0)


In [6]:
# Merge the data on state and date
df = pd.merge(covid, energy, how='inner', on=['State', 'Date'])


In [7]:
# Merge the data on state and date
df = pd.merge(energy, gdp, how='inner', on=['State', 'Date'])

In [8]:
## create long data showing sales by sector
df_long_gdp = pd.melt(df, id_vars=['State', 'Date', 'GDP'], value_vars=['Residential Sales', 'Commercial Sales', 'Industrial Sales', 'Transportation Sales'], var_name='Sector', value_name='Sales')
df_long_gdp

,State,Date,GDP,Sector,Sales
0,AK,2022-04-01,48963.4,Residential Sales,159840.67
1,AL,2022-04-01,212311.6,Residential Sales,1987430.90
2,AR,2022-04-01,125830.9,Residential Sales,1155101.80
3,AZ,2022-04-01,353565.6,Residential Sales,2403244.30
4,CA,2022-04-01,2866766.2,Residential Sales,5531397.80
...,...,...,...,...,...
9795,VT,2010-01-01,27818.0,Transportation Sales,0.00
9796,WA,2010-01-01,373940.5,Transportation Sales,530.00
9797,WI,2010-01-01,262217.8,Transportation Sales,0.00
9798,WV,2010-01-01,67289.4,Transportation Sales,348.00


In [9]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [10]:
# Total sales
df['Total Sales'] = df['Residential Sales'] + df['Commercial Sales'] + df['Industrial Sales'] + df['Transportation Sales']


In [11]:
# Data long for covid and energy sector
df_long = pd.merge(covid, energy, how='inner', on=['State', 'Date'])

df_long = pd.melt(df_long, id_vars=['State', 'Date', 'ConfirmedCases', 'StringencyIndex_WeightedAverage'], value_vars=['Residential Sales', 'Commercial Sales', 'Industrial Sales', 'Transportation Sales'], var_name='Sector', value_name='Sales')

In [12]:
unemployment = pd.read_csv("Unemployment in America Per US State.csv")

In [13]:
unemployment['Date'] = pd.to_datetime(unemployment['Year'].astype(str) + '-' + unemployment['Month'].astype(str))
features = ['State/Area', 'Date', 'Total Civilian Non-Institutional Population in State/Area']
population = unemployment[features]

In [14]:
#rename columns
population = population.rename(columns={'State/Area': 'State', 'Total Civilian Non-Institutional Population in State/Area': 'Population'})

In [15]:
# state to two letter abbreviation using dictionary

state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

population['State'] = population['State'].map(state_abbrev)

In [16]:
# Merge the data on state and date
df2 = pd.merge(df, population, how='inner', on=['State', 'Date'])

In [17]:
df2['Population'] = df2['Population'].str.replace(',', '').astype(float)

In [18]:
# Sales for each sector and GDP per capita

df2['Residential Sales per Capita'] = df2['Residential Sales'] / df2['Population']
df2['Commercial Sales per Capita'] = df2['Commercial Sales'] / df2['Population']
df2['Industrial Sales per Capita'] = df2['Industrial Sales'] / df2['Population']
df2['Transportation Sales per Capita'] = df2['Transportation Sales'] / df2['Population']
df2['GDP per Capita'] = df2['GDP'] / df2['Population']


In [19]:
#recreate long data
df_long_gdp = pd.melt(df2, id_vars=['State', 'Date', 'GDP per Capita'], value_vars=['Residential Sales per Capita', 'Commercial Sales per Capita', 'Industrial Sales per Capita', 'Transportation Sales per Capita'], var_name='Sector', value_name='Sales per Capita')


In [20]:
# filter data for 2019 to current
df_long_gdp_2019 = df_long_gdp[df_long_gdp['Date'].dt.year >= 2019]
df_long_gdp_2019.sample(5)

import us

def get_state_name_from_abbr(abbr):
    state = us.states.lookup(abbr)
    return state.name if state else None

In [21]:
df_long_gdp_2019['State'] = df_long_gdp_2019['State'].apply(get_state_name_from_abbr)
df_long_gdp_2019

C:\Users\kosti\AppData\Local\Temp\ipykernel_8692\3657969468.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_long_gdp_2019['State'] = df_long_gdp_2019['State'].apply(get_state_name_from_abbr)


,State,Date,GDP per Capita,Sector,Sales per Capita
0,Alaska,2022-04-01,0.089621,Residential Sales per Capita,0.292568
1,Alabama,2022-04-01,0.052994,Residential Sales per Capita,0.496073
2,Arkansas,2022-04-01,0.053002,Residential Sales per Capita,0.486552
3,Arizona,2022-04-01,0.060594,Residential Sales per Capita,0.411869
4,California,2022-04-01,0.092241,Residential Sales per Capita,0.177978
...,...,...,...,...,...
8031,Vermont,2019-01-01,0.056276,Transportation Sales per Capita,0.000000
8032,Washington,2019-01-01,0.088129,Transportation Sales per Capita,0.001223
8033,Wisconsin,2019-01-01,0.064760,Transportation Sales per Capita,0.000027
8034,West Virginia,2019-01-01,0.049729,Transportation Sales per Capita,0.000000


*** Interactive Scatter Plot ***

In [22]:
# Create dropdown for Sector selection, defaulting to 'Residential Sales per Capita'
sector_dropdown = alt.binding_select(options=list(df_long_gdp_2019['Sector'].unique()), name='Select Sector: ')
sector_selection = alt.selection_point(fields=['Sector'], bind=sector_dropdown, value='Residential Sales per Capita')

# Create dropdown for State selection
state_dropdown = alt.binding_select(options=list(df_long_gdp_2019['State'].unique()), name='Select State: ')
state_selection = alt.selection_point(fields=['State'], bind=state_dropdown, value='Nevada')

# Create the scatter plot with swapped axes
scatter = alt.Chart(df_long_gdp_2019).mark_circle(size=100).encode(
    x=alt.X('Sales per Capita', title='Sales per Capita', scale=alt.Scale(zero=False)),
    y=alt.Y('GDP per Capita', title='GDP per Capita', scale=alt.Scale(zero=False)),
    color=alt.Color('Sector', legend=None),  # Color by Sector
    tooltip=['Sector', 'Sales per Capita', 'GDP per Capita', 'State']
).add_params(
    sector_selection,
    state_selection
).transform_filter(
    sector_selection & state_selection
).properties(
    title='How Energy Sales Drive Economic Growth',
    width=600,
    height=400
)

# Add regression line with swapped axes
regression_line = alt.Chart(df_long_gdp_2019).transform_filter(
    sector_selection & state_selection
).transform_regression(
    'Sales per Capita', 'GDP per Capita', groupby=['Sector']
).mark_line().encode(
    x='Sales per Capita',
    y='GDP per Capita',
    color='Sector'  # Color matches scatter plot
)

# Display selected State as text on the chart
viz1_selected_state_text = alt.Chart(df_long_gdp_2019).mark_text(
    align='right',
    font='Benton Gothic, sans-serif',
    fontSize=13,
    fontWeight='normal',
    dx=-10,
    dy=10,
    color="Sector"
).encode(
    text=alt.Text('State:N')  # Column 'State' must exist in the data
).transform_filter(
    state_selection
).properties(
    width=600,  # Match the scatter plot width
    height=400  # Match the scatter plot height
).encode(
    x=alt.value(600),
    y=alt.value(375)
)

# Combine scatter plot and regression line
interactive_chart = (scatter + regression_line + viz1_selected_state_text).configure_axis(grid=False)
# Apply the 538 theme
alt.themes.enable('latimes')

interactive_chart


alt.LayerChart(...)

*** Nevada Static Scatter Plot ***

In [23]:
# Static scatter chart for Nevada and industrial energy sales

df_nevada = df_long_gdp_2019[df_long_gdp_2019['State'] == 'Nevada']
df_nevada_industrial = df_nevada[df_nevada['Sector'] == 'Industrial Sales per Capita']

scatter = alt.Chart(df_nevada_industrial).mark_circle(size=100).encode(
    x=alt.X('Sales per Capita', title='Industrial Sales per Capita', scale=alt.Scale(zero=False)),
    y=alt.Y('GDP per Capita', title='GDP per Capita', scale=alt.Scale(zero=False)),
    tooltip=['Sales per Capita', 'GDP per Capita']
).properties(
    title='How Industrial Energy Sales Drive Economic Growth in Nevada',
    width=600,
    height=400
)

regression_line = alt.Chart(df_nevada_industrial).transform_regression(
    'Sales per Capita', 'GDP per Capita'
).mark_line(color='red').encode(
    x='Sales per Capita',
    y='GDP per Capita'
)

nevada_chart = (scatter + regression_line).configure_axis(grid=False)

# Apply the 538 theme
alt.themes.enable('latimes')

nevada_chart

alt.LayerChart(...)

*** Utah Static Scatter Plot ***

In [24]:
# the same chart for utah and commercial
df_utah = df_long_gdp_2019[df_long_gdp_2019['State'] == 'Utah']
df_utah_commercial = df_utah[df_utah['Sector'] == 'Commercial Sales per Capita']

scatter = alt.Chart(df_utah_commercial).mark_circle(size=100).encode(
    x=alt.X('Sales per Capita', title='Commercial Sales per Capita', scale=alt.Scale(zero=False)),
    y=alt.Y('GDP per Capita', title='GDP per Capita', scale=alt.Scale(zero=False)),
    tooltip=['Sales per Capita', 'GDP per Capita']
).properties(
    title='How Commercial Energy Sales Drive Economic Growth in Utah',
    width=600,
    height=400
)

regression_line = alt.Chart(df_utah_commercial).transform_regression(
    'Sales per Capita', 'GDP per Capita'
).mark_line(color='red').encode(
    x='Sales per Capita',
    y='GDP per Capita'
)

# Apply the 538 theme
alt.themes.enable('latimes')

utah_chart = (scatter + regression_line).configure_axis(grid=False)

utah_chart

alt.LayerChart(...)

In [25]:
interactive_chart_html = f"<h2>How Energy Sales Drive Economic Growth</h2>\n{interactive_chart.to_html()}"
nevada_chart_html = f"<h2>How Industrial Energy Sales Drive Economic Growth in Nevada</h2>/n{nevada_chart.to_html()}"
utah_chart_html = f"<h2>How Commercial Energy Sales Drive Economic Growth in Utah</h2>/n{utah_chart.to_html()}"

In [26]:
combined_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>SI 649 - Narrative Project</title>
</head>
<body>
    <h1>SI 649 - Narrative Project</h1>
    <h4>Konstantinos Sarinopoulos(ksarinop)</h4>
    <br>
    {interactive_chart_html}
    <br>
    {nevada_chart_html}
    <br>
    {utah_chart_html}
</body>
</html>
"""

In [27]:
with open("narrative_project.html", "w") as file:
    file.write(combined_html)